In [21]:
P = Primes()
ZR.<x> = PolynomialRing(ZZ)
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
C.<x> = PolynomialRing(CC)
S = PowerSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m,x):
    f = J(n+2,m,x)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m,x): 
    f=J(n,m,x)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m,x):
    return xjNewStep(n-1,m,x)
def xjNewCoefficient(n,m,x):
    if n>0:return (xjNew(n,m,x)-xjNew(n-1,m,x))/x^n
    if n==0:return 1
def xjNewTest(n,m,x):
    return D(xjNewStep(n-1,m,x))
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def prime_n(a):return P.unrank(a-1)

def primeFactors(n):
    answer=[]
    lf=list(n.factor())
    length=len(lf)
    for k in [0..length-1]:
        pair=lf[k]
        (base,exponent)=pair
        answer=answer+[[base]]
    return(flatten(answer))

def primeFactorsInList(lst):
    ld=len(lst)
    primelist=[]
    for k in [0..ld-1]:
        a=ZZ(lst[k])
        primelist=primelist+[primeFactors(a)]
    primelist=flatten(primelist)
    primelist=list(set(primelist))
    return primelist

def den(a):
    if a in QQ:
        if a in ZZ:return 1
        return a.denominator()
def listdens(lst):
    return list(den(a) for a in lst)
def polynomialdenominator(poly):return lcm(listdens(poly))
def polynomialnumerator(poly):
    return expand(poly*polynomialdenominator(poly))
def num(a):return a.numerator()
def prime(n):
    P=Primes()
    return P.unrank(n-1)
def inclusiveListPrimesTo(n):
    if is_prime(n):
        return list(primes(n+1))
    else:
        return list(primes(n))
def chunk(k,list):
    chnk=[]
    for n in [k..len(list)-1]:
        chnk=chnk+[list[n]]
    return chnk
def drop(m,lst): return list(Set(lst).difference({m}))
def sort(list):
    answr=[]
    chnk=list
    for n in [0..len(list)-1]:
        m = min(chnk)
        answr=answr+[m]
        chnk=drop(m,chnk)
    return answr
def ord(p,n):return QQ.valuation(p)(n)
def base_conversion(number, base_from, base_to):  # by Vishnu Namboothiri
# https://ask.sagemath.org/question/9657/how-to-output-decimal-numbers-in-different-radix/
    if(base_from==10):
        return stripQuotationMarks((number.str(base=base_to)))
    else:
        decimal = Integer(str(number), base=base_from)
        return (stripQuotationMarks(decimal.str(base=base_to)))
def lastFactorWithIntegralFactors(polynomial):
    return polynomialnumerator(lastfactor(polynomial))
def numericalFactorWithIntegralFactors(polynomial):
    nf=numericalfactor(polynomial);
    pd=polynomialdenominator(lastfactor(polynomial))
    return nf/pd


def primeDivisors(n):
    pd=[]
    n=n
    fn=factor(n)
    ln=len(fn)
    for k in [0..ln-1]:
        part = fn[k]
        (prime,exponent) = part
        pd = pd + [prime]
    return pd 
def fOverField(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return ppoly

###################### following two routines disagree ####################
def factorOverField(f,prime,power):
    answer=[]
    T = GF(prime^power,'t')
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return fp

def factorOverField2(poly,prime,power,j):
    sd = leastSplittingDegree(poly,prime,bound)
    T=GF(prime^j)
    G.<t>= GF(prime^sd)
    U.<x> = T[]
    ppoly=U(poly)
    fpp=ppoly.factor()
    return fpp

############################

#######################################################
# splitting tests below fail at  e.g. f = x + 1 
############################################################

def splitsTF(f,prime,power):
    T = GF(prime^power)
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx > 1:return(False)
    return(True)

def leastSplittingDegree(f,prime,bound):
    for power in [1..bound]:
        tf=splitsTF(f,prime,power)
        if tf == True:return power
    return -1

#########################################################

def firstNprimes(n):
    P=Primes()
    first=[]
    for k in [0..n-1]:
        first=first+[P.unrank(k)]
    return first

def slopeExistsTF(pairlist):
    lenth=len(pairlist)
    if lenth<3:return False
    slopes=[]
    for k in [1..lenth-1]:
        pair1=pairlist[k-1]
        abscissa1=pair1[0]
        ordinate1=pair1[1]
        pair2=pairlist[k]
        abscissa2=pair2[0]
        ordinate2=pair2[1]
        dx=abscissa2-abscissa1
        if dx == 0:return False
        dy=ordinate2-ordinate1
        slope=dy/dx
        slopes=slopes+[slope]
    slopes=Set(slopes)
    if len(slopes) == 1:
        return [True,slopes[0]]
    if len(slopes) > 1:
        return False 
    
def zeroOfLinearMonomial(monomial,x):return x-monomial

def totalDegreeOverField(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    td=0
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        td=td+exponent
    return td

def logsAndMultiplicitiesOverField(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data

def generatingZeroLogsOverField(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:
                        if mod(i,prime)>0:
                            data=data+[i]
    data=data.sort()
    return data



def generatingZeroLogsAndMultiplicitiesOverField(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:
                        if mod(i,prime)>0:
                            data=data+[[i,exponent]]
    return data




def logsAndMultiplicitiesOverFieldTest(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    print gf
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data

def logsAndMultiplicitiesOverFieldTest(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    print gf
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data



def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data

def padListOnRightToLengthK(listo,k):
    length=len(listo)
    padded=listo
   
    if length<k:
        for j in [length..k-1]:
            padded=padded+[0]
           
        return padded
    return listo

def testCandidateZeroLogOverField(f,prime,power,candidate):
    tf=False
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    a=t^candidate
    expr3=sage_eval(expr,locals={'x':a})
    if expr3 == 0:tf=True
    return tf

def digitsToInteger(digitlist,prime):
    lenth=len(digitlist)
    sum=0
    for k in [0..lenth-1]:
        sum=sum+prime^k*digitlist[k]
    return sum

def minimumIntegerRepresentedInList(lstlst,prime):
    answer=Infinity
    ln=len(lstlst)
    for k in [0..ln-1]:
        lst=lstlst[k]
        n=digitsToInteger(lst,prime)
        if n < answer:answer=n
    return answer

def lastElement(lst):
    ln=len(lst)
    return lst[ln-1]

def push(lst):
    if len(lst)<2:return lst
    answer=[lastElement(lst)]
    for k in [0..len(lst)-2]:
        answer=flatten([answer]+[lst[k]])
    return answer

def cyclicPermutationEquivalenceClass(lst):
    if lst==[]:return lst
    answer=[lst]
    lnth=len(lst)
    for k in [0..lnth-2]:
        ple=push(lastElement(answer))
        answer=answer+[ple]
    return answer

def bestRepresentativeOfCyclicEquivalenceClass(lst,prime):
    cpec=cyclicPermutationEquivalenceClass(lst)
    minm=minimumIntegerRepresentedInList(cpec,prime)
    lcpec=len(lst) # ... = len(cpec)
    for k in [0..lcpec-1]:
        ck=cpec[k]
        dti=digitsToInteger(ck,prime)
        if minm==dti:return ck
    return "error"

def directZeroLogOverField(f,prime,power):
    tf=False
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    answer=[]
    expr3=sage_eval(expr,locals={'x':0})
    if expr3 == 0:
        answer=[0]
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            answer=answer+[i]
    return answer

def reduce(f,prime):
    poly=polynomialnumerator(D(f))
    
    ls=list(poly)
    
    def modulo(n):
        return ZZ(mod(n,prime))
    def listmod(list):
        return map(modulo,list)
    lmls = listmod(ls)
    sm=0
    for j in [0..len(lmls)-1]:
        sm=sm+lmls[j]*x^j
    return sm


#McKay-Thompson series w. a(0) = 24.
mt=[
[-1, 1],
[0, 24],
[1, 276],
[2, 2048],
[3, 11202],
[4, 49152],
[5, 184024],
[6, 614400],
[7, 1881471],
[8, 5373952],
[9, 14478180],
[10, 37122048],
[11, 91231550],
[12, 216072192],
[13, 495248952],
[14, 1102430208],
[15, 2390434947],
[16, 5061476352],
[17, 10487167336],
[18, 21301241856],
[19, 42481784514],
[20, 83300614144],
[21, 160791890304],
[22, 305854488576],
[23, 573872089212],
[24, 1063005978624],
[25,1945403602764],
[26, 3519965179904],
[27, 6300794030460],
[28, 11164248047616],
[29, 19591528119288],
[30, 34065932304384],
[31, 58718797964805],
[32, 100372723007488],
[33, 170215559855424],
[34, 286470013685760],
[35, 478625723149576],
[36, 794110053826560],
[37, 1308745319975256],
[38, 2143055278039040],
[39, 3487563372381816],
[40, 5641848336678912],
[41, 9074553043554568],
[42, 14515166263443456],
[43, 23093778743102262],
[44, 36552977852071936],
[45, 57567784186189368],
[46, 90226777113575424],
[47, 140752796480416011],
[48, 218578429975461888],
[49, 337945040343588276],
[50, 520271697765971968],
[51, 797652526220573580],
[52, 1218002527825723392],
[53, 1852604006634050072],
[54, 2807138079496716288],
[55, 4237760460302936433],
[56, 6374456847628238848],
[57, 9554873766107770560],
[58, 14273181657059143680],
[59, 21250450411204068910],
[60, 31535729115847852032],
[61, 46650835290143061624],
[62, 68797209365301886976],
[63, 101150679669913197462],
[64, 148280443106626633728],
[65, 216743142763626253712],
[66, 315923191441199824896],
[67, 459218611940943755226],
[68, 665710603285072019456],
[69, 962508846974918603904],
[70, 1388038765923851599872],
[71, 1996639069403279491427],
[72, 2864978197116521938944],
[73, 4100990608911708903432],
[74, 5856297079648098807808],
[75, 8343432715970391209502],
[76, 11859696700297921757184],
[77, 16820105145987654631552],
[78, 23802835313046730063872],
[79, 33611779636250175278886],
[80, 47362494062244172660736],
[81, 66600077798590855556532],
[82, 93460562353103053049856],
[83, 130891485964083426534122],
[84, 182952844329494181838848],
[85, 255227018229957765044016],
[86, 355376219286719031156736],
[87, 493899311443420857952845],
[88, 685157678128482627354624],
[89, 948763597225844233250504],
[90, 1311456320500974276980736],
[91, 1809633323386495729057992],
[92, 2492760414984152205361152],
[93, 3427959082742197097793024],
[94, 4706168520874397834575872],
[95, 6450411048962389429976770],
[96, 8826863296640622526464000],
[97, 12059665023346371597383976],
[98, 16450700321708764061517824],
[99, 22405985191605778318221966],
[100, 30470821082605141257437184]]

def mcKayThompson(n):return mt[n+1][1]

def sumOfRoots(f,prime,power):
    answer=0
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    #U.<x> = T[] -- this line gave an error
    poly = U(f)
    expr=str(poly)
    var('x')
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            answer=answer+a
    return answer

def productOfNonzeroRoots(f,prime,power):
    answer=1
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    #U.<x> = T[] -- this line gave an error
    poly = U(f)
    expr=str(poly)
    var('x')
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            answer=answer*a
    return answer


def coercePolynomial(f,prime,power):
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    return poly

def constantTerm(f,prime,power):
    cp=coercePolynomial(f,prime,power)
    expr=str(cp)
    var('x')
    expr3=sage_eval(expr,locals={'x':0})
    return expr3


def roots(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    expr3=sage_eval(expr,locals={'x':0})
    if expr3==0:answer=answer+[0]
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            answer=answer+[a]
    return answer

def numericalTermOverFieldNoPrint(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    xproduct=1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        xproduct=xproduct*monomial^exponent
    return g/xproduct

def polynomialDegree(poly):
    return len(poly.list())-1

def leadingCoefficient(poly):
    dp=D(poly)
    deg=polynomialDegree(dp)
    return dp.list()[deg]

def testRootLogTimesPrimeCn(f,prime,power):
    tf=True
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
    flag = 1
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            j=i*prime
            b=t^j
            expr4=sage_eval(expr,locals={'x':b})
            if expr4!=0:
                tf=False
                print (i,j)
                
    return tf

def basicRoots(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
    flag = 1
    for i in [1..prime^power-1]:
        if mod(i,prime)!=0:
            a=t^i
            expr3=sage_eval(expr,locals={'x':a})
            if expr3 == 0:answer=answer+[i]
    return answer

def basicRootsDigits(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
    flag = 1
    for i in [1..prime^power-1]:
        if mod(i,prime)!=0:
            a=t^i
            expr3=sage_eval(expr,locals={'x':a})
            if expr3 == 0:answer=answer+[padListOnRightToLengthK(i.digits(prime),power)]
    return answer

def basicLogsAndMultiplicitiesOverField(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if mod(i,prime)!=0:
                        if b^i==xm:data=data+[[i,exponent]]
    return data.sort()

def totalMultiplicityOverField(f,prime,power):
    tm=0
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:tm=tm+exponent
    return tm

def testRootLogTimesPrimeCn(f,prime,power):
    tf=True
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
    flag = 1
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            j=i*prime
            b=t^j
            expr4=sage_eval(expr,locals={'x':b})
            if expr4!=0:
                tf=False
                print (i,j)
                
    return tf


def squeezePolynomial(f,prime,power):
    dg=f.degree()
    answer=0
    for k in [0..dg]:
        j=mod(k,prime^power-1)
        cf=polynomialCoefficient(k,f)
        cf=ZZ(mod(cf,prime))
        term=ZR(cf*x^j)
        answer=answer+term
    return answer

def completeSqueezePolynomial(f,prime,power):
    sp=squeezePolynomial(f,prime,power)
    dg=sp.degree()
    answer=0
    for k in [0..dg]:
        cf=polynomialCoefficient(k,sp)
        cf=ZZ(mod(cf,prime))
        term=cf*x^k
        answer=answer+term
    print answer
    return answer

def frobenius(prime,x):return x^prime

def testDecreasing(list):
    tf=True
    ln=len(list)
    for k in [1..ln-1]:
        if list[k]>list[k-1]:tf=False
    return tf

def testIncreasing(list):
    tf=True
    ln=len(list)
    for k in [1..ln-1]:
        if list[k]<list[k-1]:tf=False
    return tf

In [ ]:
## printed output has been snipped

import pickle
wfile=open('/Users/barrybrent/run1jun21no1.txt','a')
data=[]
for m in [3..500]:
    print m
    data=data+[[m,normalJ(200,m)]]
s=pickle.dumps(str(data))
wfile.write(s)
wfile.close()
#########################
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

polydata=[]

wfile = ## printed output has been snipped

import pickle
wfile=open('/Users/barrybrent/run1jun21no1.txt','a')
data=[]
for m in [3..500]:
    print m
    data=data+[[m,normalJ(200,m)]]
s=pickle.dumps(str(data))
wfile.write(s)
wfile.close()
#########################
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

polydata=[]

wfile = open('/Users/barrybrent/run14jun21no14.txt','a') # xJ polynomials
for n in [0..200]:
    data=[]
    for k in [0..2*n+1]:
        print (n,k)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n-1,rl]] 

    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close() # print-output snippeds
for n in [0..200]:
    data=[]
    for k in [0..2*n+1]:
        print (n,k)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n-1,rl]] 

    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close() # print-output snipped

sum

In [2]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') #Jfn
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
len(s)

201

In [2]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
pairs=[]
for k in [1..200]:
    n=s[k][0]
    if (1==1)&(n>prime):
        poly=D(s[k][1])
        polynum=ZR(polynomialnumerator(D(poly)))
        sd=leastSplittingDegree(polynum,prime,bound)
        pair=(mod(n,prime),sd)
        pairs=pairs+[pair]
print sort(union(pairs))
        

[(0, 1), (1, 4), (2, 4), (3, 6), (4, 1)]


In [8]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
badns=[]
for k in [1..200]:
    n=s[k][0]
    poly=D(s[k][1])
    polynum=ZR(polynomialnumerator(D(poly)))
    polyden=ZR(polynomialdenominator(D(poly)))
    sd=leastSplittingDegree(polynum,prime,bound)
    print "---------------------------------------------------------------------"
    print "n:",n," splitting degree: ",sd
    print "denominator:";print factor(polyden)

---------------------------------------------------------------------
n: 0  splitting degree:  2
denominator:
2^3
---------------------------------------------------------------------
n: 1  splitting degree:  4
denominator:
2^10
---------------------------------------------------------------------
n: 2  splitting degree:  4
denominator:
2^7 * 3^3
---------------------------------------------------------------------
n: 3  splitting degree:  6
denominator:
2^23
---------------------------------------------------------------------
n: 4  splitting degree:  1
denominator:
2^16 * 3^3 * 5^3
---------------------------------------------------------------------
n: 5  splitting degree:  1
denominator:
2^33 * 3^6
---------------------------------------------------------------------
n: 6  splitting degree:  4
denominator:
2^29 * 3^2 * 5^3 * 7^3
---------------------------------------------------------------------
n: 7  splitting degree:  4
denominator:
2^48 * 3^6 * 5^3
----------------------------

---------------------------------------------------------------------
n: 53  splitting degree:  6
denominator:
2^321 * 3^54 * 5^32 * 7^22 * 11^12 * 13^12 * 17^9 * 19^6 * 23^6 * 29^3 * 31^3 * 37^3 * 41^3 * 43^3 * 47^3
---------------------------------------------------------------------
n: 54  splitting degree:  1
denominator:
2^316 * 3^46 * 5^35 * 7^22 * 11^15 * 13^12 * 17^9 * 19^6 * 23^6 * 29^3 * 31^3 * 37^3 * 41^3 * 43^3 * 47^3 * 53^3
---------------------------------------------------------------------
n: 55  splitting degree:  1
denominator:
2^336 * 3^53 * 5^26 * 7^25 * 11^8 * 13^12 * 17^9 * 19^6 * 23^6 * 29^3 * 31^3 * 37^3 * 41^3 * 43^3 * 47^3 * 53^3
---------------------------------------------------------------------
n: 56  splitting degree:  4
denominator:
2^322 * 3^57 * 5^35 * 7^17 * 11^15 * 13^12 * 17^9 * 19^9 * 23^6 * 29^3 * 31^3 * 37^3 * 41^3 * 43^3 * 47^3 * 53^3
---------------------------------------------------------------------
n: 57  splitting degree:  4
denominator:
2

---------------------------------------------------------------------
n: 91  splitting degree:  4
denominator:
2^549 * 3^90 * 5^57 * 7^27 * 11^24 * 13^12 * 17^15 * 19^12 * 23^12 * 29^9 * 31^6 * 37^6 * 41^6 * 43^6 * 47^3 * 53^3 * 59^3 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3
---------------------------------------------------------------------
n: 92  splitting degree:  4
denominator:
2^541 * 3^93 * 5^57 * 7^40 * 11^24 * 13^21 * 17^15 * 19^12 * 23^6 * 29^9 * 31^9 * 37^6 * 41^6 * 43^6 * 47^3 * 53^3 * 59^3 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3
---------------------------------------------------------------------
n: 93  splitting degree:  6
denominator:
2^555 * 3^90 * 5^57 * 7^40 * 11^24 * 13^21 * 17^15 * 19^12 * 23^12 * 29^9 * 31^4 * 37^6 * 41^6 * 43^6 * 47^6 * 53^3 * 59^3 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3
---------------------------------------------------------------------
n: 94  splitting degree:  1
denominator:
2^555 * 3^92 * 5^60 * 7^40 * 11^24 * 13^2

2^720 * 3^120 * 5^76 * 7^38 * 11^30 * 13^27 * 17^12 * 19^18 * 23^15 * 29^12 * 31^9 * 37^9 * 41^6 * 43^6 * 47^6 * 53^6 * 59^6 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3
---------------------------------------------------------------------
n: 120  splitting degree:  1
denominator:
2^706 * 3^115 * 5^58 * 7^53 * 11^34 * 13^27 * 17^21 * 19^18 * 23^15 * 29^12 * 31^9 * 37^9 * 41^6 * 43^6 * 47^6 * 53^6 * 59^6 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3
---------------------------------------------------------------------
n: 121  splitting degree:  4
denominator:
2^730 * 3^120 * 5^76 * 7^53 * 11^20 * 13^27 * 17^21 * 19^18 * 23^15 * 29^12 * 31^9 * 37^9 * 41^6 * 43^6 * 47^6 * 53^6 * 59^6 * 61^6 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3
---------------------------------------------------------------------
n: 122  splitting degree:  4
denominator:
2

---------------------------------------------------------------------
n: 146  splitting degree:  4
denominator:
2^870 * 3^147 * 5^93 * 7^66 * 11^40 * 13^33 * 17^24 * 19^21 * 23^18 * 29^15 * 31^12 * 37^9 * 41^9 * 43^9 * 47^9 * 53^6 * 59^6 * 61^6 * 67^6 * 71^6 * 73^2 * 79^3 * 83^3 * 89^3 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3 * 127^3 * 131^3 * 137^3 * 139^3
---------------------------------------------------------------------
n: 147  splitting degree:  4
denominator:
2^887 * 3^143 * 5^93 * 7^43 * 11^40 * 13^33 * 17^24 * 19^21 * 23^18 * 29^15 * 31^12 * 37^12 * 41^9 * 43^9 * 47^9 * 53^6 * 59^6 * 61^6 * 67^6 * 71^6 * 73^6 * 79^3 * 83^3 * 89^3 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3 * 127^3 * 131^3 * 137^3 * 139^3
---------------------------------------------------------------------
n: 148  splitting degree:  6
denominator:
2^879 * 3^147 * 5^93 * 7^66 * 11^40 * 13^33 * 17^24 * 19^21 * 23^18 * 29^15 * 31^12 * 37^6 * 41^9 * 43^9 * 47^9 * 53^6 * 59^6 * 61^6 * 67^6 * 71^6 * 73^6 *

n: 168  splitting degree:  6
denominator:
2^992 * 3^164 * 5^106 * 7^55 * 11^46 * 13^40 * 17^27 * 19^24 * 23^21 * 29^15 * 31^15 * 37^12 * 41^12 * 43^9 * 47^9 * 53^9 * 59^6 * 61^6 * 67^6 * 71^6 * 73^6 * 79^6 * 83^6 * 89^3 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3 * 127^3 * 131^3 * 137^3 * 139^3 * 149^3 * 151^3 * 157^3 * 163^3 * 167^3
---------------------------------------------------------------------
n: 169  splitting degree:  1
denominator:
2^1018 * 3^167 * 5^109 * 7^75 * 11^46 * 13^24 * 17^30 * 19^24 * 23^21 * 29^15 * 31^15 * 37^12 * 41^12 * 43^9 * 47^9 * 53^9 * 59^6 * 61^6 * 67^6 * 71^6 * 73^6 * 79^6 * 83^6 * 89^3 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3 * 127^3 * 131^3 * 137^3 * 139^3 * 149^3 * 151^3 * 157^3 * 163^3 * 167^3
---------------------------------------------------------------------
n: 170  splitting degree:  1
denominator:
2^1013 * 3^170 * 5^85 * 7^75 * 11^46 * 13^40 * 17^18 * 19^27 * 23^21 * 29^15 * 31^15 * 37^12 * 41^12 * 43^9 * 47^9 * 53^9 * 59^6 * 61^6 * 6

n: 188  splitting degree:  6
denominator:
2^1116 * 3^188 * 5^119 * 7^84 * 11^52 * 13^43 * 17^33 * 19^27 * 23^24 * 29^18 * 31^18 * 37^15 * 41^12 * 43^12 * 47^5 * 53^9 * 59^9 * 61^9 * 67^6 * 71^6 * 73^6 * 79^6 * 83^6 * 89^6 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3 * 127^3 * 131^3 * 137^3 * 139^3 * 149^3 * 151^3 * 157^3 * 163^3 * 167^3 * 173^3 * 179^3 * 181^3
---------------------------------------------------------------------
n: 189  splitting degree:  1
denominator:
2^1132 * 3^182 * 5^122 * 7^61 * 11^52 * 13^43 * 17^33 * 19^30 * 23^24 * 29^18 * 31^18 * 37^15 * 41^12 * 43^12 * 47^12 * 53^9 * 59^9 * 61^9 * 67^6 * 71^6 * 73^6 * 79^6 * 83^6 * 89^6 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3 * 127^3 * 131^3 * 137^3 * 139^3 * 149^3 * 151^3 * 157^3 * 163^3 * 167^3 * 173^3 * 179^3 * 181^3
---------------------------------------------------------------------
n: 190  splitting degree:  1
denominator:
2^1132 * 3^188 * 5^96 * 7^84 * 11^52 * 13^43 * 17^33 * 19^18 * 23^24 * 29^18 * 31^18 * 

In [4]:
def testDecreasing(list):
    tf=True
    ln=len(list)
    for k in [1..ln-1]:
        if list[k]>list[k-1]:tf=False
    return tf
        
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
badns=[]
for k in [1..20]:
    n=s[k][0]
    poly=D(s[k][1])
    polynum=ZR(polynomialnumerator(D(poly)))
    polyden=ZR(polynomialdenominator(D(poly)))
    orders=[]
    for p in [2..(n+1)]:
        if is_prime(p)==True:
            op=ord(p,polyden)
            orders=orders+[op]
    tf=testDecreasing(orders)
    if tf==False:
        badns=badns+[n]
        print factor(polyden)
print badns

2^29 * 3^2 * 5^3 * 7^3
2^58 * 3^4 * 5^6 * 7^3
2^54 * 3^8 * 5^2 * 7^3 * 11^3
2^76 * 3^15 * 5^9 * 7^2 * 11^3 * 13^3
2^96 * 3^11 * 5^4 * 7^6 * 11^3 * 13^3
[6, 9, 10, 14, 15]


In [7]:
def testDecreasing(list):
    tf=True
    ln=len(list)
    for k in [1..ln-1]:
        if list[k]>list[k-1]:tf=False
    return tf
        
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
badns=[]
for k in [1..200]:
    n=s[k][0]
    print n
    poly=D(s[k][1])
    polynum=ZR(polynomialnumerator(D(poly)))
    polyden=ZR(polynomialdenominator(D(poly)))
    orders=[]
    for p in [2..(n+1)]:
        if is_prime(p)==True:
            op=ord(p,polyden)
            orders=orders+[op]
    tf=testDecreasing(orders)
    if tf==False:
        badns=badns+[n]
        print factor(polyden)
print badns

0
1
2
3
4
5
6
2^29 * 3^2 * 5^3 * 7^3
7
8
9
2^58 * 3^4 * 5^6 * 7^3
10
2^54 * 3^8 * 5^2 * 7^3 * 11^3
11
12
13
14
2^76 * 3^15 * 5^9 * 7^2 * 11^3 * 13^3
15
2^96 * 3^11 * 5^4 * 7^6 * 11^3 * 13^3
16
17
18
19
20
2^111 * 3^21 * 5^6 * 7^9 * 11^3 * 13^3 * 17^3 * 19^3
21
2^125 * 3^18 * 5^12 * 7^4 * 11^6 * 13^3 * 17^3 * 19^3
22
2^125 * 3^20 * 5^12 * 7^9 * 11^2 * 13^3 * 17^3 * 19^3 * 23^3
23
24
25
2^154 * 3^24 * 5^8 * 7^9 * 11^6 * 13^6 * 17^3 * 19^3 * 23^3
26
2^150 * 3^27 * 5^16 * 7^9 * 11^6 * 13^2 * 17^3 * 19^3 * 23^3
27
28
2^157 * 3^27 * 5^16 * 7^5 * 11^6 * 13^6 * 17^3 * 19^3 * 23^3 * 29^3
29
30
31
32
33
2^198 * 3^29 * 5^19 * 7^12 * 11^4 * 13^6 * 17^6 * 19^3 * 23^3 * 29^3 * 31^3
34
2^198 * 3^31 * 5^22 * 7^15 * 11^9 * 13^6 * 17^2 * 19^3 * 23^3 * 29^3 * 31^3
35
2^213 * 3^36 * 5^15 * 7^8 * 11^9 * 13^6 * 17^6 * 19^3 * 23^3 * 29^3 * 31^3
36
37
38
2^221 * 3^38 * 5^22 * 7^15 * 11^9 * 13^9 * 17^6 * 19^2 * 23^3 * 29^3 * 31^3 * 37^3
39
2^237 * 3^36 * 5^25 * 7^15 * 11^9 * 13^4 * 17^6 * 19^6 * 23^3 * 29^3 * 

2^731 * 3^123 * 5^81 * 7^53 * 11^34 * 13^27 * 17^21 * 19^18 * 23^15 * 29^12 * 31^6 * 37^9 * 41^9 * 43^6 * 47^6 * 53^6 * 59^6 * 61^6 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3
125
126
127
128
129
2^774 * 3^126 * 5^84 * 7^56 * 11^34 * 13^30 * 17^21 * 19^18 * 23^15 * 29^12 * 31^12 * 37^9 * 41^9 * 43^4 * 47^6 * 53^6 * 59^6 * 61^6 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3 * 127^3
130
2^773 * 3^128 * 5^66 * 7^56 * 11^34 * 13^18 * 17^21 * 19^18 * 23^15 * 29^12 * 31^12 * 37^9 * 41^9 * 43^9 * 47^6 * 53^6 * 59^6 * 61^6 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3 * 127^3 * 131^3
131
132
2^780 * 3^128 * 5^84 * 7^59 * 11^25 * 13^30 * 17^21 * 19^21 * 23^15 * 29^12 * 31^12 * 37^9 * 41^9 * 43^9 * 47^6 * 53^6 * 59^6 * 61^6 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3 * 127^3 * 131^3
133
2^797 * 3^132 * 5^84 * 7^42 * 11^37 *

2^1062 * 3^175 * 5^113 * 7^78 * 11^49 * 13^40 * 17^30 * 19^27 * 23^21 * 29^18 * 31^15 * 37^12 * 41^12 * 43^12 * 47^9 * 53^9 * 59^9 * 61^6 * 67^6 * 71^6 * 73^6 * 79^6 * 83^6 * 89^2 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3 * 127^3 * 131^3 * 137^3 * 139^3 * 149^3 * 151^3 * 157^3 * 163^3 * 167^3 * 173^3 * 179^3
179
180
181
182
2^1083 * 3^183 * 5^116 * 7^59 * 11^49 * 13^29 * 17^30 * 19^27 * 23^21 * 29^18 * 31^15 * 37^12 * 41^12 * 43^12 * 47^9 * 53^9 * 59^9 * 61^9 * 67^6 * 71^6 * 73^6 * 79^6 * 83^6 * 89^6 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3 * 127^3 * 131^3 * 137^3 * 139^3 * 149^3 * 151^3 * 157^3 * 163^3 * 167^3 * 173^3 * 179^3 * 181^3
183
2^1104 * 3^180 * 5^116 * 7^81 * 11^49 * 13^43 * 17^30 * 19^27 * 23^24 * 29^18 * 31^15 * 37^12 * 41^12 * 43^12 * 47^9 * 53^9 * 59^9 * 61^4 * 67^6 * 71^6 * 73^6 * 79^6 * 83^6 * 89^6 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3 * 127^3 * 131^3 * 137^3 * 139^3 * 149^3 * 151^3 * 157^3 * 163^3 * 167^3 * 173^3 * 179^3 * 181^3
184
2^1090 * 3^182 

In [11]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
o2=[]
for k in [1..200]:
    n=s[k][0]
    if is_prime(n)==True:
        poly=D(s[k][1])
        polynum=ZR(polynomialnumerator(D(poly)))
        polyden=ZR(polynomialdenominator(D(poly)))
        o2=o2+[ord(2,polyden)]
print o2


[7, 23, 33, 48, 71, 81, 105, 119, 142, 177, 192, 225, 249, 263, 288, 321, 359, 369, 407, 432, 441, 479, 503, 537, 585, 609, 624, 647, 657, 681, 764, 791, 825, 839, 897, 911, 945, 983, 1007, 1041, 1079, 1089, 1150, 1161, 1185, 1198]


In [18]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
o3=[]
for k in [1..200]:
    n=s[k][0]
    if is_prime(n)==True:
        poly=D(s[k][1])
        polynum=ZR(polynomialnumerator(D(poly)))
        polyden=ZR(polynomialdenominator(D(poly)))
        o3=o3+[ord(3,polyden)]
print o3



[3, 0, 6, 6, 11, 12, 16, 18, 23, 30, 30, 36, 42, 42, 48, 54, 60, 60, 66, 72, 72, 78, 83, 90, 96, 101, 102, 108, 108, 114, 126, 132, 138, 138, 150, 150, 156, 162, 168, 173, 179, 180, 191, 192, 198, 198]


In [21]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
o5=[]
for k in [1..200]:
    oldpolyden=1
    n=s[k][0]
    if is_prime(n)==True:
        poly=D(s[k][1])
        polynum=ZR(polynomialnumerator(D(poly)))
        polyden=ZR(polynomialdenominator(D(poly)))
        quotient=ZR(polyden/oldpolyden)
        oldpolyden=polyden
        print "==============================================================="
        print "n: ",n
        print factor(quotient)




n:  2
2^7 * 3^3
n:  3
2^23
n:  5
2^33 * 3^6
n:  7
2^48 * 3^6 * 5^3
n:  11
2^71 * 3^11 * 5^6 * 7^3
n:  13
2^81 * 3^12 * 5^6 * 7^6 * 11^3
n:  17
2^105 * 3^16 * 5^9 * 7^6 * 11^3 * 13^3
n:  19
2^119 * 3^18 * 5^12 * 7^6 * 11^3 * 13^3 * 17^3
n:  23
2^142 * 3^23 * 5^12 * 7^9 * 11^6 * 13^3 * 17^3 * 19^3
n:  29
2^177 * 3^30 * 5^19 * 7^12 * 11^6 * 13^6 * 17^3 * 19^3 * 23^3
n:  31
2^192 * 3^30 * 5^19 * 7^12 * 11^6 * 13^6 * 17^3 * 19^3 * 23^3 * 29^3
n:  37
2^225 * 3^36 * 5^22 * 7^15 * 11^9 * 13^6 * 17^6 * 19^6 * 23^3 * 29^3 * 31^3
n:  41
2^249 * 3^42 * 5^25 * 7^18 * 11^9 * 13^9 * 17^6 * 19^6 * 23^3 * 29^3 * 31^3 * 37^3
n:  43
2^263 * 3^42 * 5^25 * 7^18 * 11^12 * 13^9 * 17^6 * 19^6 * 23^3 * 29^3 * 31^3 * 37^3 * 41^3
n:  47
2^288 * 3^48 * 5^28 * 7^18 * 11^12 * 13^9 * 17^6 * 19^6 * 23^6 * 29^3 * 31^3 * 37^3 * 41^3 * 43^3
n:  53
2^321 * 3^54 * 5^32 * 7^22 * 11^12 * 13^12 * 17^9 * 19^6 * 23^6 * 29^3 * 31^3 * 37^3 * 41^3 * 43^3 * 47^3
n:  59
2^359 * 3^60 * 5^38 * 7^25 * 11^15 * 13^12 * 17^9 * 19^9 * 23^

n:  197
2^1185 * 3^198 * 5^125 * 7^88 * 11^55 * 13^46 * 17^33 * 19^30 * 23^24 * 29^18 * 31^18 * 37^15 * 41^12 * 43^12 * 47^12 * 53^9 * 59^9 * 61^9 * 67^6 * 71^6 * 73^6 * 79^6 * 83^6 * 89^6 * 97^6 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3 * 127^3 * 131^3 * 137^3 * 139^3 * 149^3 * 151^3 * 157^3 * 163^3 * 167^3 * 173^3 * 179^3 * 181^3 * 191^3 * 193^3
n:  199
2^1198 * 3^198 * 5^129 * 7^88 * 11^55 * 13^46 * 17^33 * 19^30 * 23^24 * 29^18 * 31^18 * 37^15 * 41^12 * 43^12 * 47^12 * 53^9 * 59^9 * 61^9 * 67^6 * 71^6 * 73^6 * 79^6 * 83^6 * 89^6 * 97^6 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3 * 127^3 * 131^3 * 137^3 * 139^3 * 149^3 * 151^3 * 157^3 * 163^3 * 167^3 * 173^3 * 179^3 * 181^3 * 191^3 * 193^3 * 197^3


In [10]:
def testDecreasing(list):
    tf=True
    ln=len(list)
    for k in [1..ln-1]:
        if list[k]>list[k-1]:tf=False
    return tf
        
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
badns=[]
for k in [1..200]:
    n=s[k][0]
    if is_prime(n)==True:
        poly=D(s[k][1])
        polynum=ZR(polynomialnumerator(D(poly)))
        polyden=ZR(polynomialdenominator(D(poly)))
        orders=[]
        for p in [2..(n+1)]:
            if is_prime(p)==True:
                op=ord(p,polyden)
                orders=orders+[op]
        tf=testDecreasing(orders)
        if tf==False:
            badns=badns+[n]
            print factor(polyden)
print badns

[]


In [5]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
def mod3(n):return mod(n,3)
for k in [1..200]:
    n=s[k][0]
    if is_prime(n)==True:
        poly=D(s[k][1])
        polynum=ZR(polynomialnumerator(D(poly)))
        polyden=ZR(polynomialdenominator(D(poly)))
        orders=[]
        for p in [2..(n+1)]:
            if is_prime(p)==True:
                op=ord(p,polyden)
                orders=orders+[op]
        print orders
        print map(mod3,orders)
        print "--------------------------------------------------------------------------------------------"
    

[7, 3]
[1, 0]
--------------------------------------------------------------------------------------------
[23, 0]
[2, 0]
--------------------------------------------------------------------------------------------
[33, 6, 0]
[0, 0, 0]
--------------------------------------------------------------------------------------------
[48, 6, 3, 0]
[0, 0, 0, 0]
--------------------------------------------------------------------------------------------
[71, 11, 6, 3, 0]
[2, 2, 0, 0, 0]
--------------------------------------------------------------------------------------------
[81, 12, 6, 6, 3, 0]
[0, 0, 0, 0, 0, 0]
--------------------------------------------------------------------------------------------
[105, 16, 9, 6, 3, 3, 0]
[0, 1, 0, 0, 0, 0, 0]
--------------------------------------------------------------------------------------------
[119, 18, 12, 6, 3, 3, 3, 0]
[2, 0, 0, 0, 0, 0, 0, 0]
--------------------------------------------------------------------------------------------
[142

[1089, 180, 116, 81, 49, 43, 30, 27, 21, 18, 15, 12, 12, 12, 9, 9, 9, 6, 6, 6, 6, 6, 6, 6, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0]
[0, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
--------------------------------------------------------------------------------------------
[1150, 191, 122, 84, 52, 43, 33, 30, 24, 18, 18, 15, 12, 12, 12, 9, 9, 9, 6, 6, 6, 6, 6, 6, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0]
[1, 2, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
--------------------------------------------------------------------------------------------
[1161, 192, 122, 84, 52, 43, 33, 30, 24, 18, 18, 15, 12, 12, 12, 9, 9, 9, 6, 6, 6, 6, 6, 6, 6, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0]
[0, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [22]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
o5=[]
for k in [1..200]:
    oldpolyden=1
    n=s[k][0]
    poly=D(s[k][1])
    polynum=ZR(polynomialnumerator(D(poly)))
    polyden=ZR(polynomialdenominator(D(poly)))
    quotient=ZR(polyden/oldpolyden)
    oldpolyden=polyden
    print "==============================================================="
    print "n: ",n
    print factor(quotient)



n:  0
2^3
n:  1
2^10
n:  2
2^7 * 3^3
n:  3
2^23
n:  4
2^16 * 3^3 * 5^3
n:  5
2^33 * 3^6
n:  6
2^29 * 3^2 * 5^3 * 7^3
n:  7
2^48 * 3^6 * 5^3
n:  8
2^37 * 3^8 * 5^3 * 7^3
n:  9
2^58 * 3^4 * 5^6 * 7^3
n:  10
2^54 * 3^8 * 5^2 * 7^3 * 11^3
n:  11
2^71 * 3^11 * 5^6 * 7^3
n:  12
2^62 * 3^9 * 5^6 * 7^3 * 11^3 * 13^3
n:  13
2^81 * 3^12 * 5^6 * 7^6 * 11^3
n:  14
2^76 * 3^15 * 5^9 * 7^2 * 11^3 * 13^3
n:  15
2^96 * 3^11 * 5^4 * 7^6 * 11^3 * 13^3
n:  16
2^82 * 3^15 * 5^9 * 7^6 * 11^3 * 13^3 * 17^3
n:  17
2^105 * 3^16 * 5^9 * 7^6 * 11^3 * 13^3
n:  18
2^103 * 3^12 * 5^9 * 7^6 * 11^3 * 13^3 * 17^3 * 19^3
n:  19
2^119 * 3^18 * 5^12 * 7^6 * 11^3 * 13^3 * 17^3
n:  20
2^111 * 3^21 * 5^6 * 7^9 * 11^3 * 13^3 * 17^3 * 19^3
n:  21
2^125 * 3^18 * 5^12 * 7^4 * 11^6 * 13^3 * 17^3 * 19^3
n:  22
2^125 * 3^20 * 5^12 * 7^9 * 11^2 * 13^3 * 17^3 * 19^3 * 23^3
n:  23
2^142 * 3^23 * 5^12 * 7^9 * 11^6 * 13^3 * 17^3 * 19^3
n:  24
2^131 * 3^20 * 5^16 * 7^9 * 11^6 * 13^3 * 17^3 * 19^3 * 23^3
n:  25
2^154 * 3^24 * 5^8 * 7^9 

n:  80
2^465 * 3^80 * 5^39 * 7^34 * 11^21 * 13^18 * 17^12 * 19^12 * 23^9 * 29^6 * 31^6 * 37^6 * 41^3 * 43^3 * 47^3 * 53^3 * 59^3 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3
n:  81
2^490 * 3^72 * 5^51 * 7^34 * 11^21 * 13^18 * 17^12 * 19^12 * 23^9 * 29^6 * 31^6 * 37^6 * 41^6 * 43^3 * 47^3 * 53^3 * 59^3 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3
n:  82
2^486 * 3^80 * 5^51 * 7^34 * 11^21 * 13^18 * 17^12 * 19^12 * 23^9 * 29^6 * 31^6 * 37^6 * 41^2 * 43^3 * 47^3 * 53^3 * 59^3 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3
n:  83
2^503 * 3^83 * 5^51 * 7^37 * 11^21 * 13^18 * 17^12 * 19^12 * 23^9 * 29^6 * 31^6 * 37^6 * 41^6 * 43^3 * 47^3 * 53^3 * 59^3 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3
n:  84
2^491 * 3^81 * 5^54 * 7^25 * 11^21 * 13^18 * 17^15 * 19^12 * 23^9 * 29^6 * 31^6 * 37^6 * 41^6 * 43^3 * 47^3 * 53^3 * 59^3 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3
n:  85
2^512 * 3^82 * 5^41 * 7^37 * 11^21 * 13^18 * 17^8 * 19^12 * 23^9 * 29^6 * 31^6 * 37^6 * 41^6 * 43^6 * 47^3 * 53^3 * 59^3 * 61^3 * 67^3 * 71^3 * 73^3 * 79^3 * 8

n:  127
2^764 * 3^126 * 5^81 * 7^56 * 11^34 * 13^27 * 17^21 * 19^18 * 23^15 * 29^12 * 31^12 * 37^9 * 41^9 * 43^6 * 47^6 * 53^6 * 59^6 * 61^6 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3
n:  128
2^745 * 3^129 * 5^81 * 7^56 * 11^34 * 13^27 * 17^21 * 19^18 * 23^15 * 29^12 * 31^12 * 37^9 * 41^9 * 43^9 * 47^6 * 53^6 * 59^6 * 61^6 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3 * 127^3
n:  129
2^774 * 3^126 * 5^84 * 7^56 * 11^34 * 13^30 * 17^21 * 19^18 * 23^15 * 29^12 * 31^12 * 37^9 * 41^9 * 43^4 * 47^6 * 53^6 * 59^6 * 61^6 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3 * 127^3
n:  130
2^773 * 3^128 * 5^66 * 7^56 * 11^34 * 13^18 * 17^21 * 19^18 * 23^15 * 29^12 * 31^12 * 37^9 * 41^9 * 43^9 * 47^6 * 53^6 * 59^6 * 61^6 * 67^3 * 71^3 * 73^3 * 79^3 * 83^3 * 89^3 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3 * 127^3 * 131^3
n:  131
2^791 * 3^132 * 5^84 * 7^56 * 11^37 * 13

n:  160
2^942 * 3^157 * 5^81 * 7^72 * 11^43 * 13^36 * 17^27 * 19^24 * 23^21 * 29^15 * 31^15 * 37^12 * 41^9 * 43^9 * 47^9 * 53^9 * 59^6 * 61^6 * 67^6 * 71^6 * 73^6 * 79^6 * 83^3 * 89^3 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3 * 127^3 * 131^3 * 137^3 * 139^3 * 149^3 * 151^3 * 157^3
n:  161
2^964 * 3^162 * 5^103 * 7^53 * 11^43 * 13^36 * 17^27 * 19^24 * 23^12 * 29^15 * 31^15 * 37^12 * 41^9 * 43^9 * 47^9 * 53^9 * 59^6 * 61^6 * 67^6 * 71^6 * 73^6 * 79^6 * 83^3 * 89^3 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3 * 127^3 * 131^3 * 137^3 * 139^3 * 149^3 * 151^3 * 157^3
n:  162
2^967 * 3^152 * 5^103 * 7^72 * 11^43 * 13^36 * 17^27 * 19^24 * 23^21 * 29^15 * 31^15 * 37^12 * 41^9 * 43^9 * 47^9 * 53^9 * 59^6 * 61^6 * 67^6 * 71^6 * 73^6 * 79^6 * 83^3 * 89^3 * 97^3 * 101^3 * 103^3 * 107^3 * 109^3 * 113^3 * 127^3 * 131^3 * 137^3 * 139^3 * 149^3 * 151^3 * 157^3 * 163^3
n:  163
2^983 * 3^162 * 5^103 * 7^72 * 11^43 * 13^36 * 17^27 * 19^24 * 23^21 * 29^15 * 31^15 * 37^12 * 41^12 * 43^9 * 47^9 * 53^9

In [2]:
def testDecreasing(list):
    tf=True
    ln=len(list)
    for k in [1..ln-1]:
        if list[k]>list[k-1]:tf=False
    return tf
        
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
one=[]
zero=[]
badns=[]
for k in [1..200]:
    n=s[k][0]
   
    poly=D(s[k][1])
    polynum=ZR(polynomialnumerator(D(poly)))
    polyden=ZR(polynomialdenominator(D(poly)))
    md2=mod(polyden,2)
    if md2==1:one=one+[n]
    if md2==0:zero=zero+[n]
print "one:";print one;print "zero:";print zero

one:
[]
zero:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199]


In [23]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
differences=[]
for k in [1..20]:
    n=s[k][0]
    if is_prime(n)==True:
        poly=D(s[k][1])
        polynum=ZR(polynomialnumerator(D(poly)))
        polyden=ZR(polynomialdenominator(D(poly)))
        np=next_prime(lastElement(prime_divisors(polyden)))
        print (n,np)
        print (n,ZR(mod(polyden,np)))
        print "--------------------------------------------------------------"

(2, 5)
(2, 1)
--------------------------------------------------------------
(3, 3)
(3, 2)
--------------------------------------------------------------
(5, 5)
(5, 3)
--------------------------------------------------------------
(7, 7)
(7, 6)
--------------------------------------------------------------
(11, 11)
(11, 5)
--------------------------------------------------------------
(13, 13)
(13, 12)
--------------------------------------------------------------
(17, 17)
(17, 2)
--------------------------------------------------------------
(19, 19)
(19, 6)
--------------------------------------------------------------


In [24]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..200]:
    n=s[k][0]
    
    poly=D(s[k][1])
  
    polyden=ZR(polynomialdenominator(D(poly)))
    lep=lastElement(prime_divisors(polyden))
    np=next_prime(lep)
    mnp=ZR(mod(polyden,np))
    data=data+[mnp]
    print (n,mnp)
print data

(0, 2)
(1, 1)
(2, 1)
(3, 2)
(4, 2)
(5, 3)
(6, 3)
(7, 6)
(8, 5)
(9, 10)
(10, 4)
(11, 5)
(12, 12)
(13, 12)
(14, 7)
(15, 9)
(16, 5)
(17, 2)
(18, 6)
(19, 6)
(20, 18)
(21, 6)
(22, 18)
(23, 17)
(24, 19)
(25, 28)
(26, 21)
(27, 5)
(28, 25)
(29, 19)
(30, 7)
(31, 21)
(32, 8)
(33, 31)
(34, 10)
(35, 12)
(36, 19)
(37, 3)
(38, 18)
(39, 22)
(40, 24)
(41, 30)
(42, 16)
(43, 24)
(44, 28)
(45, 17)
(46, 47)
(47, 41)
(48, 3)
(49, 22)
(50, 23)
(51, 1)
(52, 11)
(53, 39)
(54, 54)
(55, 15)
(56, 54)
(57, 56)
(58, 16)
(59, 16)
(60, 62)
(61, 27)
(62, 29)
(63, 57)
(64, 64)
(65, 25)
(66, 37)
(67, 61)
(68, 3)
(69, 6)
(70, 18)
(71, 63)
(72, 66)
(73, 54)
(74, 30)
(75, 72)
(76, 55)
(77, 37)
(78, 34)
(79, 77)
(80, 19)
(81, 65)
(82, 6)
(83, 80)
(84, 12)
(85, 82)
(86, 68)
(87, 87)
(88, 33)
(89, 22)
(90, 84)
(91, 27)
(92, 4)
(93, 72)
(94, 12)
(95, 48)
(96, 87)
(97, 32)
(98, 87)
(99, 52)
(100, 90)
(101, 90)
(102, 92)
(103, 81)
(104, 32)
(105, 84)
(106, 9)
(107, 49)
(108, 5)
(109, 63)
(110, 79)
(111, 66)
(112, 125)
(113, 48)

In [25]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..200]:
    n=s[k][0]
    
    poly=D(s[k][1])
  
    polyden=ZR(polynomialdenominator(D(poly)))
    lep=lastElement(prime_divisors(polyden))
    np=next_prime(lep)
    mnp=ZR(mod(polyden,np))
    if is_prime(n)==True:
        print (n,mnp)
        data=data+[mnp]
print data

(2, 1)
(3, 2)
(5, 3)
(7, 6)
(11, 5)
(13, 12)
(17, 2)
(19, 6)
(23, 17)
(29, 19)
(31, 21)
(37, 3)
(41, 30)
(43, 24)
(47, 41)
(53, 39)
(59, 16)
(61, 27)
(67, 61)
(71, 63)
(73, 54)
(79, 77)
(83, 80)
(89, 22)
(97, 32)
(101, 90)
(103, 81)
(107, 49)
(109, 63)
(113, 48)
(127, 52)
(131, 43)
(137, 97)
(139, 66)
(149, 11)
(151, 6)
(157, 31)
(163, 109)
(167, 125)
(173, 100)
(179, 61)
(181, 167)
(191, 179)
(193, 7)
(197, 100)
(199, 111)
[1, 2, 3, 6, 5, 12, 2, 6, 17, 19, 21, 3, 30, 24, 41, 39, 16, 27, 61, 63, 54, 77, 80, 22, 32, 90, 81, 49, 63, 48, 52, 43, 97, 66, 11, 6, 31, 109, 125, 100, 61, 167, 179, 7, 100, 111]


In [15]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..200]:
    n=s[k][0]
    
    poly=D(s[k][1])
  
    polyden=ZR(polynomialdenominator(D(poly)))
    lep=lastElement(prime_divisors(polyden))
    np=next_prime(lep)
    mnp=mod(polyden,np)
    if is_prime(n)==False:
        print (n,mnp)
        data=data+[mnp]
print data

(0, 2)
(1, 1)
(4, 2)
(6, 3)
(8, 5)
(9, 10)
(10, 4)
(12, 12)
(14, 7)
(15, 9)
(16, 5)
(18, 6)
(20, 18)
(21, 6)
(22, 18)
(24, 19)
(25, 28)
(26, 21)
(27, 5)
(28, 25)
(30, 7)
(32, 8)
(33, 31)
(34, 10)
(35, 12)
(36, 19)
(38, 18)
(39, 22)
(40, 24)
(42, 16)
(44, 28)
(45, 17)
(46, 47)
(48, 3)
(49, 22)
(50, 23)
(51, 1)
(52, 11)
(54, 54)
(55, 15)
(56, 54)
(57, 56)
(58, 16)
(60, 62)
(62, 29)
(63, 57)
(64, 64)
(65, 25)
(66, 37)
(68, 3)
(69, 6)
(70, 18)
(72, 66)
(74, 30)
(75, 72)
(76, 55)
(77, 37)
(78, 34)
(80, 19)
(81, 65)
(82, 6)
(84, 12)
(85, 82)
(86, 68)
(87, 87)
(88, 33)
(90, 84)
(91, 27)
(92, 4)
(93, 72)
(94, 12)
(95, 48)
(96, 87)
(98, 87)
(99, 52)
(100, 90)
(102, 92)
(104, 32)
(105, 84)
(106, 9)
(108, 5)
(110, 79)
(111, 66)
(112, 125)
(114, 52)
(115, 117)
(116, 57)
(117, 62)
(118, 39)
(119, 109)
(120, 83)
(121, 19)
(122, 75)
(123, 29)
(124, 121)
(125, 117)
(126, 110)
(128, 77)
(129, 22)
(130, 122)
(132, 135)
(133, 10)
(134, 35)
(135, 8)
(136, 115)
(138, 88)
(140, 53)
(141, 62)
(142, 139)
(143

In [33]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..200]:
    n=s[k][0]
    
    poly=D(s[k][1])
  
    polyden=ZR(polynomialdenominator(D(poly)))
    lep=lastElement(prime_divisors(polyden))
    np=next_prime(lep)
    mnp=ZR(mod(polyden,np))
    if is_prime(n)==True:
        print (n,(polyden-mnp)/np)
        op=ord(np,polyden-mnp)
        data=data+[op]
print;print;print
print data

(2, 691)
(3, 2796202)
(5, 1252412463513)
(7, 3664201036108361142)
(11, 203790978680349290447455697454545)
(13, 241839744134372807784211190827994820923076)
(17, 69018750134899264369920366379585600874299692694588235294)
(19, 5592186370172205070547382733716676256132199661341893901473684210526)
(23, 29487404940291107594603207314957935483549386391742756110405506016361111195826086956521)
(29, 1258817403740702842298756459942362638759595375844748797060914467080263755661310183906358958851177931034482758620689)
(31, 941115597735608898200774740821018890376912501751120690073246576181243802913771787623098667431045427321558709677419354838709)
(37, 282875388451413119889563205918771766144730981512626016162190981751687576189513108734416888278263094282917250226568370867977729647291641081081081081081081081)
(41, 14897034279487052809038631710469172447292279893837706068058629352805977955142780120466629187777966954406256319306462855616883747572766400286338546283123387317073170731707317073170)
(43, 213483730

(167, 1947239912244769577349001105198696401872121130373825275468017385759544846997932451345061723360433118237309597480952240888475592133696674838589312593096243393017019944801640674723577329652930541625180957454395907222708112901677447717402587162620027498653001093592591436666051153374581565408605015226176285326810880059084575915617453205154687304952953482059611811209443300676374896705679589273428142946460311107020996490863251878281867720030925165484631743100504378762476717640349802819033711317762940862549662177060031336835662286641054316133932726129141319332240464039519799943544255689680743479971548407670836837578979696083986222526529365704195806775415210573446512194312605909016385930611384349611293130741849831275624669612013705902186583947091140295365019255240574986157471421951160947399758317322284181569408282023015917985088810378699561233159882868056943406437676392872697042652076284627544910179640718562874251497005988023952095808383233532934131736526946107784431137724550898203592814

In [32]:
ord(2,8)

3